# 4 - Example - NSRDB data to model NRELs bifacial field


<a id='step1'></a>

In [1]:
from pathlib import Path
import os

testfolder = Path().resolve().parent.parent / 'bifacialvf' / 'TEMP' / 'BARN'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)

In [3]:
import pandas as pd
import bifacialvf

In [4]:
# This information helps with debugging and getting support :)
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacialVF version ", bifacialvf.__version__)

Working on a  Windows 10
Python version  3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]
Pandas version  2.1.0
bifacialVF version  0.1.8.1+0.g539ebce.dirty


In [5]:
NREL_API_KEY = None  # <-- please set your NREL API key here
# note you must use "quotes" around your key as it is a string.

if NREL_API_KEY is None:
       NREL_API_KEY = 'DEMO_KEY'  # OK for this demo, but better to get your own key

In [6]:
import pvlib

# NREL Coords: 39.7407° N, 105.1686° W

metdata, metadata = pvlib.iotools.get_psm3(
    latitude=39.7407, longitude=-105.1686,
    api_key=NREL_API_KEY,
    email='silvana.ovaitt@nrel.gov',  # <-- any email works here fine
    names='2022', map_variables=True, leap_day=False)

C:\Users\sayala\Documents\GitHub\pvlib-python\pvlib\iotools\psm3.py:183: pvlibDeprecationWarning: The ``get_psm3`` function will default to leap_day=True starting in pvlib 0.11.0. Specify leap_day=True to enable this behavior now, or specify leap_day=False to hide this warning.
  warnings.warn(


# 3. Set Parameters

In [7]:
# Variables
sazm = 180                  # PV Azimuth(deg) or tracker axis direction
cw = 2 # m . 1-up portrait
pitch=5.7
hub_height=1.5 # m
norm_hub_height = 1.5/cw
norm_pitch = 5.7/cw
rowType = "interior"        
sensorsy = 12                # sampling areas in the module. edges would be 1 and 12.
PVfrontSurface = "glass"    # options: glass or ARglass
PVbackSurface = "glass"     # options: glass or ARglass

# Tracking instructions
tracking=True
backtrack=True
limit_angle = 50

deltastyle = 'exact'  # NSRDB downloads data at center hour 11:30, 12:30, etc... 

In [8]:
# We have outdated names that do not match NSRDB. renaming.
#name in NSRDB : name in bifacialVF
metdata.rename(columns={"dni": "DNI", "dhi": "DHI", 'temp_air':'DryBulb', 'wind_speed':'VWind'}, inplace=True)
metadata['TZ'] = metadata['Time Zone']
metadata['city'] = metadata['City']


In [9]:
writefiletitle = os.path.join(testfolder, 'Dirk_2022_Results.csv')
bifacialvf.simulate(metdata, metadata, writefiletitle=writefiletitle, 
            sazm=sazm, pitch=norm_pitch, hub_height=norm_hub_height, 
         rowType=rowType, sensorsy=sensorsy, 
         PVfrontSurface=PVfrontSurface, PVbackSurface=PVbackSurface, 
          tracking=tracking, backtrack=backtrack, 
         limit_angle=limit_angle, deltastyle=deltastyle)



Calculating Sun position with no delta, for exact timestamp in input Weather File
No albedo value set or included in TMY3 file (TMY Column name 'Alb (unitless)' expected) Setting albedo default to 0.2
 
 
********* 
Running Simulation for TMY3: 
Location:   -
Lat:  39.73  Long:  -105.18  Tz  -7
Parameters: tilt:  0   Sazm:  180     Hub_Height :  1.5   Pitch:  5.7   Row type:  interior   Albedo:  0.2
Saving into Dirk\Dirk_2022_Results.csv
 
 
Distance between rows for no shading on Dec 21 at 9 am solar time =  0.0
Actual distance between rows =  4.7
 
 ***** IMPORTANT --> THIS SIMULATION Has Tracking Activated
Backtracking Option is set to:  True


100%|██████████| 8760/8760 [00:41<00:00, 209.96it/s]

Finished
